In [135]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import copy

In [136]:
def displayChart(tab):
    Chart=[]
    columns = []
    N = len(tab)-1
    for i in range(N, -1, -1):
        chart = []
        columns.append(N-i)
        for j in range(0,i):
            chart.append("")
        for j in range(i, N+1):
            entry = tab[j][i]
            chart.append(entry)
        Chart.append(chart)
    df = pd.DataFrame(Chart, columns = columns)
    return df.style.hide_index()

In [137]:
def list_to_dict(L):
    res = {}
    for i in range(len(L)):
        res[i] = L[i]
    return res

In [138]:
def tree(t, init, up, down, negative_allowed):
    """t=num of time periods. init=initial value. up, down factors : num, negative values allowed : bool."""
    res = {}
    gap = up+down
    if negative_allowed:
        for i in range(t+1):
            res[i] = np.arange(init-i*down, init+(i+1)*up, gap)
    else:
        for i in range(t+1):
            res[i] = np.maximum(np.zeros(i+1), np.arange(init-i*down, init+(i+1)*up, gap))
    return res

In [139]:
def bond_tree(bond):
    print("unimplemented")
    L = []
    for i in range(bond.periods):
        pass

In [140]:
class HoLee:
    def __init__(self, periods, R0, lam, sig):
        self.periods = periods
        self.R0 = R0
        self.lam = lam
        self.sig = sig
        self.tree_exists = False

    def create_tree(self):
        if self.lam+self.sig > 0:
            self.up = self.lam+self.sig
            self.down = self.sig-self.lam
        else:
            self.up = self.sig-self.lam
            self.down = self.lam+self.sig
        
        self.tree = tree(self.periods, self.R0, self.up, self.down, True)
        self.tree_exists = True
        return self.tree

In [141]:
class Stocks:
    def __init__(self, periods, S0, alpha, beta):
        self.periods = periods
        self.S0 = S0
        self.alpha = alpha
        self.up = self.alpha
        self.beta = beta
        self.down = self.beta
        self.tree_exists = False

    def create_tree(self):
        self.tree = tree(self.periods, self.S0, self.alpha, self.beta, False)
        self.tree_exists = True
        return self.tree

In [142]:
def p_values(model):
    t = model.periods
    stock_price_chart = model.stocks_tree
    rates = model.rates_tree
    P = {i: np.ones(i+1) for i in range (t+1)}

    # P = {i: [1]*(i+1) for i in range(t+1)}
    for i in range(0, t):
        for j in range(i+1):
            if stock_price_chart[i][j] == 0:
                P[i][j] = -1
            else:
                d = stock_price_chart[i+1][j] / stock_price_chart[i][j]
                u = stock_price_chart[i+1][j+1] / stock_price_chart[i][j]
                P[i][j] = (1 + rates[i][j] - d) / (u-d)
    model.stock_p_vals = P
    return P

In [143]:
def rate_p_values(model):
    t = model.periods
    res = {}
    for i in range(t):
        res[i] = np.full(i+1, 0.5)
    res[t] = np.ones(t+1)
    model.rate_p_vals = res
    return res

In [144]:
class Model(HoLee, Stocks):
    """Model inherits from HoLee and Stocks. Creation of trees are delayed via methods to address efficiency concerns."""
    def __init__(self, periods):
        self.periods = periods
    
    def init_stocks(self, S0, alpha, beta):
        stocks = Stocks(self.periods, S0, alpha, beta)
        self.stock_up = alpha
        self.stock_down = beta
        self.stocks_tree = stocks.create_tree()
        self.p_values = p_values(self)

    def init_HoLee(self, R0, lam, sig):
        rates = HoLee(self.periods, R0, lam, sig)
        if lam+sig > 0:
            self.rate_up = lam+sig
            self.rate_down = sig-lam
        else:
            self.rate_up = sig-lam
            self.rate_down = lam+sig
        self.rates_tree = rates.create_tree()
        self.rate_p_values = rate_p_values(self)

    
    # def init_bond(self, face, q, distr_list, conv_list, call_list):
    #     bond = Bond(self.periods, face, q, distr_list, conv_list, call_list)
    #     self.bond_tree = bond.create_tree

In [145]:
class Bond(Model):
    def __init__(self, periods, face, q, distr_list, conv_list, call_list):
        super().__init__(periods)
        self.face = face
        self.q = q
        self.coup = self.face * self.q
        self.distr_dict = list_to_dict(distr_list)
        self.conv_dict = list_to_dict(conv_list)
        self.call_dict = list_to_dict(call_list)
    
    def create_tree(self):
        if not HoLee.tree_exists:
            HoLee.create_tree
        if not Stocks.tree_exists:
            Stocks.create_tree
        self.tree = bond_tree(self)
        return self.tree


In [146]:
model = Model(4)
model.init_HoLee(0.04, 0, 0.005)
print("interest rates")
displayChart(model.rates_tree)

interest rates


/var/folders/bw/yjjm_7gs7vv11d3rbv60z3p00000gn/T/ipykernel_43420/4140021945.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4
,,,,0.060000
,,,0.055000,0.050000
,,0.050000,0.045000,0.040000
,0.045000,0.040000,0.035000,0.030000
0.040000,0.035000,0.030000,0.025000,0.020000


In [147]:
model.init_stocks(30,5,5)
print("stocks")
displayChart(model.stocks_tree)

stocks


/var/folders/bw/yjjm_7gs7vv11d3rbv60z3p00000gn/T/ipykernel_43420/4140021945.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4
,,,,50.000000
,,,45.000000,40.000000
,,40.000000,35.000000,30.000000
,35.000000,30.000000,25.000000,20.000000
30.000000,25.000000,20.000000,15.000000,10.000000


In [148]:
# distr_list = [(10, 0) for i in range(11)]
# distr_list[10] = (10, 0, 0.5)
# conv_list = [1 for i in range(11)]
# call_list = [0 for i in range(11)]
# model.init_bond(50, .04, distr_list, conv_list, call_list)

In [149]:
p_vals = p_values(model)
print("p tilde")
displayChart(p_vals)
# print(p_vals)
# print(p_vals[1][1])

p tilde


/var/folders/bw/yjjm_7gs7vv11d3rbv60z3p00000gn/T/ipykernel_43420/4140021945.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4
,,,,1.000000
,,,0.747500,1.000000
,,0.700000,0.657500,1.000000
,0.657500,0.620000,0.587500,1.000000
0.620000,0.587500,0.560000,0.537500,1.000000


In [150]:
rate_p_vals = rate_p_values(model)
print("rate p tilde")
displayChart(rate_p_vals)

rate p tilde


/var/folders/bw/yjjm_7gs7vv11d3rbv60z3p00000gn/T/ipykernel_43420/4140021945.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4
,,,,1.000000
,,,0.500000,1.000000
,,0.500000,0.500000,1.000000
,0.500000,0.500000,0.500000,1.000000
0.500000,0.500000,0.500000,0.500000,1.000000


In [151]:
distr_list = [(10, 0) for i in range(11)]
distr_list[10] = (10, 0, 0.5)
conv_list = np.ones(11) #[1 for i in range(11)]
call_list = np.zeros(11) #[0 for i in range(11)]
cbond = Bond(4, 50, 0.04, distr_list, conv_list, call_list)

In [152]:
"""
Idea: one dictionary. Key: integer tuple (time period, # stock heads, # tails), Value: bond price before coupon.
"""

def two_coin(model, bond):
    face = 50
    t = model.periods
    S = model.stocks_tree
    R = model.rates_tree
    s_p = model.p_vals
    r_p = model.rate_p_vals
    c = bond.coup
    res = {}
    #fill in the last period 
    for i in range(t+1):            #i: number of stock heads
        for j in range(t+1):        #j: number of rate heads
            cur_stock = S[t][i]
            res[(t, i, j)] = max(conv_list[t]*cur_stock, face * (bond.distr_list[t+1][2] if cur_stock <= bond.distr_list[t+1][0] else 1))
    #now all the final period prices have been filled. what remains is to perform backwards induction
    #Backwards Induction:
    for n in range(t-1, -1, -1):
        for i in range(n+1):        #i: number of stock heads
            for j in range(n+1):    #j: number of rate heads
                cur_stock = S[n][i]
                cur_rate = R[n][i]
                cur_disc = 1/(1+cur_rate)
                #there are four cases. two adjacent stock price moves and two adjacent rates moves. make sure to get the correct pvals
                #stock head rate head
                hh = res[(n+1, i+1, j+1)]
                hh_coup = 
                #stock head rate tail

                #stock tail rate head

                #stock tail rate tail

                res[(n, i, j)]
        pass



print(model.rate_up)
    

0.005
